In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plot_utils as u
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
result_dir = Path("long_pi_grid")
lam = 0
line_colors = ['#dcc48e', '#eaefd3', '#b3c0a4', '#505168', '#27233a', '#561f37', '#b3c0a4', '#505168', '#27233a']
setpoint = 1.0414E-4
tail_length = 20000

In [ ]:
kp_list = set()
ti_list = set()
seed_list = set()

for simdir in result_dir.iterdir():
    if not simdir.is_dir():
        continue
    outfiles = list(simdir.glob('*.out'))
    if len(outfiles) != 1:
        continue
    for output_file in outfiles:
        config = u.read_config_from_output_file(output_file)
        seed_list.add(config['RandomSeed'])
        kp_list.add(config['kp'])
        ti_list.add(config['ti'])

In [ ]:
fitness = dict()
for ti in ti_list:
    fitness[ti] = {
        kp: []
        for kp in kp_list
    }

In [ ]:
for simdir in result_dir.iterdir():
    if not simdir.is_dir():
        continue
    outfiles = list(simdir.glob('*.out'))
    if len(outfiles) != 1:
        continue
    for output_file in outfiles:
        config = u.read_config_from_output_file(output_file)
        seed = config['RandomSeed']
        kp = config['kp']
        ti = config['ti']
        
    controller_t, controller_p, controller_b = u.load_controller_data(simdir)
    controller_e = (controller_b - config['setpoint']) / config['setpoint']
    dt = controller_t[1] = controller_t[0]
    
    num_samples = int(tail_length / dt)
    j = np.sum((controller_e[-num_samples:]) ** 2 + lam * (controller_p[-num_samples:] ** 2)) / (2 * num_samples)
    fitness[ti][kp].append((seed, j))

In [ ]:
color_number = 0
for ti in fitness:
    ti_data = fitness[ti]
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_facecolor("white")
    mean_line = []
    for kp in ti_data:
        j_data = ti_data[kp]
        if len(j_data) == 0:
            continue
        j = np.array([e[1] for e in j_data])
        j_mean = np.mean(j)
        j_std = np.std(j)
        mean_line.append((kp, j_mean))
        plt.plot([kp, kp], [j_mean - j_std, j_mean + j_std], color="black")
    kp_all, j_mean = map(list, zip(*sorted(mean_line, key=lambda e: e[0])))
    plt.plot(kp_all, j_mean, color=line_colors[color_number])
        
    u.format_single_2d_plot(fig, "Kp", "Fitness", f"Fitness for Ti={ti}, lambda={lam}")
    fig.savefig(f"fitness_data_ti_{ti:.1f}.png", bbox_inches="tight")